In [19]:
!pip install adaptkeybert datasets transformers sentence-transformers pandas keybert scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
import pandas as pd
df_sentences = pd.read_csv("preprocessed_sentences.csv")

In [9]:
df_sentences

,Unnamed: 0,document_id,paragraph_id,sentence_id,sentence
0,0,1,1,1,"Q4 was solid, highlighted by 32 billings growt..."
1,1,1,1,2,One of the highlights on the call was the earl...
2,2,1,1,3,Another standout on the call was the number of...
3,3,1,2,1,While F16 rev guide was only tweaked up mildly...
4,4,1,2,2,"Encouragingly, management appeared committed t..."
...,...,...,...,...,...
255,255,7,6,1,We were impressed by the demo of the Silevo pi...
256,256,7,6,2,We believe these technologies will help SCTY r...
257,257,7,7,1,New subsidy programs or new states where Solar...
258,258,7,8,1,Leadership position could become a greater bar...


In [6]:
# Version 3.0 (Based on unsupervised list of LLaMA)

#To Revise: sorted by priorities
"""
- Product
- Technology
- Leadership
- Customer
- Research & Development (more updates)
"""

keywords_dict = {
    "Financial Performance": {
        "EPS": ["performance indicators","Earnings per Share expectations","price-to-earnings ratio"],
        "Cash Flow":["free cash flow","liquidity","working capital","cash reserves"],
        "Revenue": ["operational revenue", "business income", "operating profit","increased income","sales growth rate","earnings trajectory"],
        "Return On Equity": ["capital efficiency", "investment return", "capital ROI","ROE"],
        "Margins": ["EBITDA margins", "operating margin", "profit margin", "cost-to-revenue", "gross profit","profitability changes"],
        "Cost Management": ["expense control", "cost reduction", "efficiency savings","expense trimming", "cost containment","cost optimization","cost-effectiveness"],
        "Dividend Policy": ["dividend payout", "shareholder returns", "yield", "dividend sustainability", "payout ratio"],
        "Investments": ["strategic investment", "capital deployment", "fundraising", "fund allocation"],
        "Balance Sheet": ["assets", "liabilities", "equity", "debt management", "financial health","book value","debt-to-equity ratio","equity issuance","capital structure"],
    },
    "Company": {
        "Long-term Growth": ["sustainable growth", "long-term trajectory", "future growth","scalability", "large-scale expansion","unit expansion","unit volume"],
        "Mergers & Acquisition": ["business acquisition", "M&A activity", "buyout approach","merger"],
        "Refranchising": ["franchise model", "franchise transitions", "refranchising plans"],
        "Sustainability": ["green initiatives", "environmental impact", "sustainable practices","ESG","SRI","sustainability","green energy adoption","social responsibility"],
        "Employees": ["workforce optimization","talent management","upskilling","remote work","employee management","employee benefits","diversity and inclusion","salary levels"],
        "Research & Development":["R&D spending","patent activity","technological breakthrough","future of work","continuous development","research projects"],
        "Marketing":["brand awareness","CI","corporate identity","performance marketing","brand loyalty","word-of-mouth","brand value","consumer perception"],
        "Shares Repurchase": ["buyback", "repurchase programs", "shareholder value", "equity reduction", "stock repurchases","stock rebuy"],
        "Processes":["process improvements","streamlined processes","productivity improvements","operational efficiency"],
        "Leadership":["effective leadership", "executive strength", "executive resilience","management trust","crisis management","risk assessment","contingency planning"],
    },
    "Product": {
        "Innovation": ["new features", "innovative products", "product advancements","new product","product launch","disruptive technology","portfolio diversification"],
        "Product Characteristics":["USP","unique selling point","product quality", "product differentiation","product portfolio"],
        "Pricing Strategy":["price segmentation","price optimization", "dynamic pricing", "competitive pricing","pricing models", "price elasticity", "discount strategies"],
        "Production": ["production capacities","manufacturing delays", "supply issues","production stops","factory problems","logistics bottlenecks","material shortage"],
        "Technology Trends":["autonomous systems","IoT (Internet of Things)", "machine learning","deep learning","natural language processing","AI","robotics","digital transformation","cloud computing","blockchain"]
    },
    "Market": {
        "Market Share": ["market share", "industry share", "market proportion","market penetration"],
        "Market Expansion": ["new markets", "geographical reach", "market entry","worldwide expansion"],
        "Competitors":["market rivalry", "competitive threats", "industry competition", "competitive advantage"],
        "Global Presence": ["international footprint", "global operations", "worldwide coverage"],
        "Industry Outlook":["sector growth","market trends","market evolution","industry trends"],
		"Regulations":["tax regulations","regulatory risks","governmental influence","government incentives", "state funding", "subsidies","political influence","legal disputes"],
        "Partnerships and Collaborations":["strategic alliances","partner relationships","joint venture"],
        "Supply Chain":["logistics optimization", "supply logistics", "supply chain strategies","supply constraints","inventory challenges","distribution channels", "supplier relationships","procurement"],
        "Economic Conditions": ["economic environment", "market economy", "macroeconomic factors","recession","expansion","inflationary impact","interest rate environment","foreign exchange impact"],
        "Demand":["increasing demand","decreasing demand","demand forecasting","consumer visits","store traffic"],
        "Customer": ["user interaction", "customer retention","customer loyalty","frequent buyer","user satisfaction","customer lifetime value (CLV)","per-visit spending","churn rate"]
    }
}

In [20]:
from adaptkeybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import pipeline


# Training

# Testing

In [ ]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Initialize KeyBERT and SentenceTransformer
kw_model = KeyBERT()
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize FinancialBERT for sentiment analysis
sentiment_model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
financial_bert_sentiment = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=tokenizer)

# Flatten the keywords into a single list for zero-shot KeyBERT
seed_words = [
    keyword
    for category, arguments in keywords_dict.items()
    for argument, keywords in arguments.items()
    for keyword in keywords
]

# Precompute embeddings for each argument's seed_words
precomputed_seed_embeddings = {
    argument: sentence_transformer.encode(seed_words)
    for category, arguments in keywords_dict.items()
    for argument, seed_words in arguments.items()
}

# Define the classification and analysis function
def classify_and_analyze(sentence, kw_model, keywords_dict, precomputed_seed_embeddings, seed_words, threshold=0.65):
    # Extract keywords with KeyBERT
    keywords = kw_model.extract_keywords(
        sentence, top_n=5, seed_keywords=seed_words
    )
    extracted_keywords = [kw[0] for kw in keywords]

    # Initialize results
    categories = []
    for category, arguments in keywords_dict.items():
        for argument, seed_words in arguments.items():
            # Use precomputed embeddings for the argument
            seed_embeddings = precomputed_seed_embeddings[argument]
            # Encode extracted keywords
            keyword_embeddings = sentence_transformer.encode(extracted_keywords)
            # Compute cosine similarities
            similarities = cosine_similarity(keyword_embeddings, seed_embeddings)
            max_similarity = similarities.max() if similarities.size > 0 else 0
            if max_similarity >= threshold:
                categories.append(argument)
    
    # Use FinancialBERT to classify sentiment
    sentiment_result = financial_bert_sentiment(sentence)[0]
    sentiment = sentiment_result['label']

    return categories, sentiment

# Prepare the DataFrame for processing
df_sentences['arguments'] = None
df_sentences['sentiment'] = None

# Apply the function to classify sentences and analyze sentiment
for idx, row in df_sentences.iterrows():
    categories, sentiment = classify_and_analyze(
        row['sentence'], kw_model, keywords_dict, precomputed_seed_embeddings, seed_words
    )
    df_sentences.at[idx, 'arguments'] = categories
    df_sentences.at[idx, 'sentiment'] = sentiment



"""# Test the function on individual sentences
for sentence in df_sentences['sentence']:
    categories, sentiment = classify_and_analyze(sentence, kw_model, keywords_dict, precomputed_seed_embeddings, seed_words)
    print(f"Sentence: {sentence}\nCategories: {categories}, Sentiment: {sentiment}\n")
"""

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'# Test the function on individual sentences\nfor sentence in df_sentences[\'sentence\']:\n    categories, sentiment = classify_and_analyze(sentence, kw_model, keywords_dict, precomputed_seed_embeddings, seed_words)\n    print(f"Sentence: {sentence}\nCategories: {categories}, Sentiment: {sentiment}\n")\n'

In [27]:
df_sentences

,Unnamed: 0,document_id,paragraph_id,sentence_id,sentence,arguments,sentiment
0,0,1,1,1,"Q4 was solid, highlighted by 32 billings growt...","[Long-term Growth, Industry Outlook]",positive
1,1,1,1,2,One of the highlights on the call was the earl...,"[Market Share, Market Expansion, Industry Outl...",positive
2,2,1,1,3,Another standout on the call was the number of...,"[Investments, Market Share, Competitors, Indus...",neutral
3,3,1,2,1,While F16 rev guide was only tweaked up mildly...,[],positive
4,4,1,2,2,"Encouragingly, management appeared committed t...","[Margins, Long-term Growth, Employees, Processes]",positive
...,...,...,...,...,...,...,...
255,255,7,6,1,We were impressed by the demo of the Silevo pi...,[Technology Trends],positive
256,256,7,6,2,We believe these technologies will help SCTY r...,"[Research & Development, Market Share, Market ...",positive
257,257,7,7,1,New subsidy programs or new states where Solar...,[Regulations],neutral
258,258,7,8,1,Leadership position could become a greater bar...,[Leadership],neutral


In [ ]:
# Initialize AdaptKeyBERT and SentenceTransformer
kw_model = KeyBERT()
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

precomputed_seed_embeddings = {
    argument: sentence_transformer.encode(seed_words)
    for category, arguments in keywords_dict.items()
    for argument, seed_words in arguments.items()
}

# Flattening the keywords into a single list
seed_words = [
    argument 
    for category, arguments in keywords_dict.items() 
    for argument, keywords in arguments.items() 
    for keyword in keywords
]

def classify_sentence(sentence, kw_model, keywords_dict, precomputed_seed_embeddings, seed_words, threshold=0.65):
    # Extract keywords from the sentence using KeyBERT (zero-shot training)
    keywords = kw_model.extract_keywords(
        sentence, top_n=5, seed_keywords=seed_words, nr_candidates=50
    )
    extracted_keywords = [kw[0] for kw in keywords]

    results = {}
    for category, arguments in keywords_dict.items():
        argument_scores = {}
        for argument, seed_words in arguments.items():
            # Use precomputed embeddings for the argument
            seed_embeddings = precomputed_seed_embeddings[argument]
            # Encode the extracted keywords
            keyword_embeddings = sentence_transformer.encode(extracted_keywords)
            # Compute cosine similarities
            similarities = cosine_similarity(keyword_embeddings, seed_embeddings)
            max_similarity = similarities.max() if similarities.size > 0 else 0
            if max_similarity >= threshold:
                argument_scores[argument] = max_similarity
        
        if argument_scores:
            results[category] = argument_scores
    
    return results if results else {"Uncategorized": "No relevant category found"}

# Global list of seed_words for KeyBERT zero-shot training
seed_words = [
    keyword
    for category, arguments in keywords_dict.items()
    for argument, keywords in arguments.items()
    for keyword in keywords
]

# Test the function on sentences
for sentence in df_sentences['sentence']:
    categories = classify_sentence(sentence, kw_model, keywords_dict, precomputed_seed_embeddings, seed_words)
    print(f"Sentence: {sentence}\nCategories: {categories}\n")


Sentence: Q4 was solid, highlighted by 32 billings growth yy, 80.00 which suggests the health of fundamentals and secular trends.
Categories: {'Company': {'Long-term Growth': 0.7785666}, 'Market': {'Industry Outlook': 0.7731363}}

Sentence: One of the highlights on the call was the early success of recently introduced Wave Analytics, including wins at marquee logos such as Time Warner Cable, Merck, and others, which suggests the credibility of the product cyclegrowth engine against what looks like a massive multi billion dollar market opportunity.
Categories: {'Uncategorized': 'No relevant category found'}

Sentence: Another standout on the call was the number of SaaSApplication Software 8figure deals, which likely reflects the companys industry vertical positioning, the strategic nature of relationships and comprehensive platform strategy.
Categories: {'Financial Performance': {'Investments': 0.7786581}, 'Market': {'Market Share': 0.66499156, 'Competitors': 0.764099, 'Industry Outlook